In [ ]:
!pip install datasets
!pip install evaluate
!pip install peft
!pip install accelerate -U


In [ ]:
pip install numpy --upgrade

In [41]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [42]:
# how dataset was generated

# load recipes data
recipes_dataset = load_dataset("Thefoodprocessor/recipe_new_with_features_full")

# define subsample size train
N = 1000
# generate indexes for random subsample
rand_idx = np.random.randint(24999, size=N)

# define subsample size for test
N = 200
# generate indexes for random subsample
rand_idx2 = np.random.randint(50000, size=N)


In [48]:
# extract train and test data

x_train = recipes_dataset['train'][rand_idx]['recipe_new']
y_train = recipes_dataset['train'][rand_idx]['meal_type']

for i in range(0,len(y_train)):
    y_train[i] = y_train[i].split(',')[0] #manually split, cuz all have multiple categories depending on the occasion

x_test = recipes_dataset['train'][rand_idx2]['recipe_new'] #there is only train dataset so use this
y_test = recipes_dataset['train'][rand_idx2]['meal_type'] #there is only train dataset so use this

for i in range(0,len(y_test)):
    y_test[i] = y_test[i].split(',')[0] #manually split, cuz all have multiple categories depending on the occasion


# print(y_train[0])
# print(y_train[0].split(',')[0])
# print('\n')
# print(y_train[1])
# print(y_train[1].split(',')[0])
# print('\n')
#print(y_train)
# create new dataset


In [49]:
recipes_dataset["train"].column_names

['recipe_original',
 'title_original',
 'title_cleaned',
 'recipe_new',
 'wine_type',
 'allergy_type',
 'diet_type',
 'holiday',
 'cuisine_type',
 'meal_type',
 'ingredients_alternatives']

In [50]:
all_labels = recipes_dataset['train'][0:]['meal_type']
all_labels2 = []
for i in range(0,len(all_labels)):
    pseduo_list = all_labels[i].split(',')
    for items in pseduo_list:
        all_labels2.append(items) #manually split, cuz all have multiple categories depending on the occasion

all_labels = list(set(all_labels2))
    
print(len(all_labels))
print(all_labels)


159
['', 'Tea', ' SaladPlease provide', ' DinnerPlease let', ' RussianPlease let me', ' Southern', ' FilipinoPlease', ' ItalianPlease let me', ' PicnicPlease provide', ' Christmas', ' SnackPlease provide', ' Roast', ' Pie', ' Entree', ' Dess', ' Juice', ' Jam', ' SnackPlease', ' Party Food', ' Halloween', 'Breakfast', ' Latin AmericanPlease let', ' Dim Sum', ' Snack', ' Tea', 'Brunch', ' Appet', ' JapanesePlease let', ' Taco', ' BBQ', ' LunchPlease', ' Asian', ' PizzaPlease provide', ' Wine', 'Pasta', 'Dinner', ' LunchPlease provide', ' FishPlease let me', ' Slovakian', 'Mexican', ' Beverage', ' GreekPlease provide', ' Party', ' Dinner', ' SoupPlease let', 'Beverage', ' D', ' Soup', ' Tapas', 'Middle Eastern', 'Japanese', ' Tacos', ' Brunch', ' Main Course', 'Entrée', ' Lunch', ' Breakfast', 'Bread', ' Seafood', ' Pizza', ' KoreanPlease provide the', ' Pastry', 'Snack', ' DinnerPlease', ' SaladPlease let', ' Street Food', ' Korean', ' Sauce', ' Vietnamese', ' Quick Meal', ' Dessert', '

In [51]:
model_checkpoint = 'distilbert-base-uncased'
# model_checkpoint = 'roberta-base' # you can alternatively use roberta-base but this model is bigger thus training will take longer

# Define your class names and IDs
class_ids = range(0,len(all_labels))

# define label maps
id2label = {}
for name, idx in zip(all_labels, class_ids):
    id2label[idx] = name

label2id = {}
for name, idx in zip(all_labels, class_ids):
    label2id[name] = idx

for keys in id2label:
    label_test = id2label[keys]
    id_test = label2id[label_test]
    id2label[id_test]

#run this once only
for i in range(0, len(y_train)):
    y_train[i] = label2id[y_train[i]]

for i in range(0, len(y_test)):
    y_test[i] = label2id[y_test[i]]

print(y_test)
print(y_train)
print(type(y_train[0]))
print(type(y_test[0]))

dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
                             'validation':Dataset.from_dict({'label':y_test,'text':x_test})})

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=159, id2label=id2label, label2id=label2id)

[129, 83, 133, 35, 129, 78, 151, 129, 35, 35, 35, 129, 133, 35, 35, 35, 35, 20, 129, 133, 20, 35, 78, 35, 35, 35, 75, 35, 35, 129, 148, 129, 20, 129, 35, 129, 35, 129, 129, 20, 35, 20, 35, 35, 148, 129, 35, 35, 20, 129, 129, 129, 129, 35, 129, 129, 35, 129, 129, 129, 35, 35, 129, 35, 20, 20, 35, 35, 35, 129, 129, 35, 20, 75, 83, 133, 35, 20, 133, 129, 35, 133, 129, 35, 35, 129, 35, 20, 35, 35, 129, 129, 35, 54, 129, 78, 35, 35, 35, 35, 35, 129, 78, 20, 78, 35, 35, 35, 35, 129, 129, 20, 35, 129, 35, 20, 35, 133, 35, 62, 133, 129, 133, 133, 20, 133, 35, 35, 35, 133, 35, 20, 35, 133, 129, 129, 35, 35, 35, 133, 35, 148, 35, 133, 129, 129, 148, 35, 35, 35, 35, 35, 35, 35, 129, 35, 35, 129, 133, 133, 129, 129, 35, 35, 35, 35, 35, 129, 35, 35, 35, 133, 83, 133, 20, 129, 35, 129, 20, 35, 35, 35, 129, 35, 129, 129, 129, 129, 35, 35, 83, 35, 35, 35, 35, 20, 133, 35, 35, 129]
[35, 35, 35, 83, 20, 20, 35, 35, 35, 129, 35, 133, 129, 129, 35, 35, 129, 35, 35, 35, 133, 133, 35, 129, 35, 129, 35, 129,

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
# display architecture
model


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [53]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [68]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        padding = True,
        max_length=512
    )
    return tokenized_inputs



In [69]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
#tokenized_datasets = tokenized_dataset.remove_columns(recipes_dataset["train"].column_names)
tokenized_dataset["train"]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [70]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [71]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [72]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}


In [ ]:
# define list of examples
#?
#dessert
#dessert
#dinner
#brekky
text_list = ["""Pan-Seared Sole with Mustard Sauce and Zucchini FrittersRice Almandine with Cranberries 3 sole fillets Salt and pepper. 
            Olive oil, for sauteing White Wine Mustard Sauce, recipe follows Green olives, chopped Zucchini Fritters, recipe follows 
            Rice Almandine, recipe follows 1 cup white wine 1 cup yellow mustard 1 cup heavy cream ...3 zucchini Salt 1 egg, 
            beaten 1/2 cup Italian bread crumbs 1 clove garlic, minced Vegetable oil, for sauteing 1 cup long grain rice 
            1/8 cup dried cranberries 1/4 cup slivered almonds 1 tablespoon butter Instructions: 
            1. Remove skin from sole fillets with a knife. 
            2. Season each fillet with salt and pepper on both sides. 
            3. Heat olive oil in a large saute pan over high heat. Sear sole fillets until slightly brown and cooked through. 
            4. Top each fillet with white wine mustard sauce and diced green olives. 
            5. Reduce wine by half in a medium-sized saute pan over medium-high heat. Add mustard and cream, stirring until combined. 
            6. Clean and grate zucchini, placing it in a colander to drain water. Salt to taste. 
            7. Mix zucchini with egg, breadcrumbs, and garlic in a medium-sized bowl. Toss until coated with breadcrumb mixture.  
            8. Form patties and cook in a large saute pan over medium-high heat until browned and crispy. 
            9. Bring 2 cups of water to a rolling boil in a small pot. Add rice and reduce heat to low. Simmer until all water absorbed. 
            10. Saute cranberries and almonds in butter over medium heat in a small pan. Add to rice before serving.""", 
            
             """Spiced Fruit Tart Pie dough 1/2 pound dried figs 1/4 pound raisins 1/2 pint red wine 
             1 pound cooking apples 1 pound pears 4 ounces caster sugar 1 tablespoon flaked almonds 
             1/2 teaspoon powdered cloves 1/2 teaspoon ground mace 1/2 teaspoon powdered cinnamon 
             1/2 teaspoon powdered ginger Instructions: 
             1. Preheat oven to 350 degrees F. 
             2. Simmer figs and raisins in wine for 30 minutes. 
             3. Peel, core, and slice apples and pears. 
             4. Add the sugar, almonds, and spices to the figs and raisins and simmer until the apples and pears are soft. 
             5. Line a dish with pie dough. 
             6. Fill with the fruit mixture and bake for 1/2 hour or until pastry is cooked.""", 
             
             """Lemon Ice with Ginger and Lemon Cups 8 medium to large lemons 1 1/2 cups sugar 2 cups water 2-inch piece fresh ginger, 
             peeled and sliced Crushed ice 1 lemon, zest finely grated Fresh mint leaves, for garnish Instructions: 
             1. To make the lemon cups, trim a thin slice off the bottom of each lemon so they can stand upright. 
             2. Hollow out the insides of the lemons using a teaspoon, taking care not to damage the lemon shell. 
             3. Reserve the chopped lemon pulp for later use. 
             4. Freeze the lemon cups in an egg carton for at least 1 hour or overnight. 
             5. To make the ginger syrup, combine sugar, water, and ginger in a pot over medium heat. 
             6. Simmer and stir for 2 minutes until the sugar is dissolved and the ginger is fragrant. 
             7. Remove from heat, cover, and steep until cool.
             8. Fill a blender with crushed ice and pour in the reserved lemon juice, adding the ginger 
             syrup after removing the big pieces of ginger. 
             9. Blend until slushy and freeze for 1 hour to firm up. 
             10. Scoop the lemon ginger ice into the prepared lemon cups and garnish with mint leaves before serving.""", 
             
             """Tuna Provençal 5 tablespoons extra-virgin olive oil 1 teaspoon ground cumin, plus additional 1 teaspoon fennel seeds, 
             plus additional 1 clove garlic, peeled and chopped 2 sprigs thyme, leaves only 1 lime, zest finely grated Freshly ground 
             black pepper 2 (1-pound) tuna steaks 2 Texas onions (spring onions), peeled, trimmed, and each cut into 8 wedges 1 bulb fennel, 
             trimmed and cut into 1/4-inch thick slices 2 small zucchini, trimmed and cut into 1/4-inch-thick slices 2 orange baby bell peppers,
             cored, seeded, deveined, and cut into 1/4-inch thick slices Salt 12 fava beans, peeled 8 cherry tomatoes, halved 1 seedless orange,
             peeled and cut in segments 1 lemon, zest finely grated, flesh peeled, and cut in segments 4 sprigs basil, leaves only, 
             washed 2 tablespoons Nicoise olives, pitted 1 tablespoon pistachios Vegetable oil Fleur de sel Instructions: 
             1. Combine olive oil, cumin, fennel seeds, garlic, thyme leaves, and lime zest in a bowl. Season with pepper to taste. 
             Rub the marinade all over tuna steaks and refrigerate for at least 30 minutes or up to 2 hours. 
             2. Heat olive oil in a large skillet over medium heat. Cook onions, fennel, cumin, and fennel seeds for 
             5 minutes without browning. Add zucchini and bell peppers, season with salt and pepper, 
             and continue cooking for another 5 minutes. 
             3. Heat additional olive oil in a small skillet over medium heat. Cook fava beans for 1 minute,
             then add cherry tomatoes, orange segments, lemon zest and segments, half of the basil leaves, olives, and pistachios. 
             4. Grill tuna steaks over high heat until medium rare, about 4 minutes per side. Let rest 
             for 5 minutes before slicing into 4 pieces. 5. Divide vegetables among 4 dinner plates and
             top each plate with remaining basil leaves and 2 tuna slices. Spoon citrus-fava mixture around the plate and serve immediately.""", 
            
             """Eggs with Peas and Shallots 2 tablespoons butter 1 large shallot, very thinly sliced 8 eggs Salt and freshly ground 
             black pepper 1 package soft herb-flavored cheese, crumbled (recommended: Boursin) 1 cup green peas Instructions: 
             1. Melt butter in a medium skillet over medium heat. 
             2. Add shallots and cook for 3-4 minutes. 
             3. Whisk eggs with salt and pepper and combine with shallots. 
             4. Stir in the cheese and peas. 
             5. Cook for 3-4 minutes until eggs are firming up but not yet dry. 
             6. Serve immediately."""]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
  
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + "\n - " + id2label[predictions.tolist()])

In [74]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [75]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [76]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 749,727 || all params: 67,825,470 || trainable%: 1.105376785446529


In [85]:
# hyperparameters
lr = 1e-3
batch_size = 32
num_epochs = 50

In [86]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [87]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

C:\Users\david\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.013934,{'accuracy': 0.74}
2,No log,1.069617,{'accuracy': 0.755}
3,No log,1.077127,{'accuracy': 0.745}
4,No log,1.108084,{'accuracy': 0.725}
5,No log,1.069414,{'accuracy': 0.755}
6,No log,1.335335,{'accuracy': 0.745}
7,No log,1.308660,{'accuracy': 0.735}
8,No log,1.326262,{'accuracy': 0.765}
9,No log,1.580281,{'accuracy': 0.74}
10,No log,1.428895,{'accuracy': 0.745}


Trainer is attempting to log a value of "{'accuracy': 0.74}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.755}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.745}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.725}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.755}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This in

TrainOutput(global_step=1600, training_loss=0.06488829486072063, metrics={'train_runtime': 3514.0297, 'train_samples_per_second': 14.229, 'train_steps_per_second': 0.455, 'total_flos': 6757072588800000.0, 'train_loss': 0.06488829486072063, 'epoch': 50.0})

In [84]:
model.to('cpu') # moving to mps for Mac (can alternatively do 'cpu')

print("Trained model predictions:")
print("--------------------------")
# define list of examples
text_list = ["""Pan-Seared Sole with Mustard Sauce and Zucchini FrittersRice Almandine with Cranberries 3 sole fillets Salt and pepper. 
            Olive oil, for sauteing White Wine Mustard Sauce, recipe follows Green olives, chopped Zucchini Fritters, recipe follows 
            Rice Almandine, recipe follows 1 cup white wine 1 cup yellow mustard 1 cup heavy cream ...3 zucchini Salt 1 egg, 
            beaten 1/2 cup Italian bread crumbs 1 clove garlic, minced Vegetable oil, for sauteing 1 cup long grain rice 
            1/8 cup dried cranberries 1/4 cup slivered almonds 1 tablespoon butter Instructions: 
            1. Remove skin from sole fillets with a knife. 
            2. Season each fillet with salt and pepper on both sides. 
            3. Heat olive oil in a large saute pan over high heat. Sear sole fillets until slightly brown and cooked through. 
            4. Top each fillet with white wine mustard sauce and diced green olives. 
            5. Reduce wine by half in a medium-sized saute pan over medium-high heat. Add mustard and cream, stirring until combined. 
            6. Clean and grate zucchini, placing it in a colander to drain water. Salt to taste. 
            7. Mix zucchini with egg, breadcrumbs, and garlic in a medium-sized bowl. Toss until coated with breadcrumb mixture.  
            8. Form patties and cook in a large saute pan over medium-high heat until browned and crispy. 
            9. Bring 2 cups of water to a rolling boil in a small pot. Add rice and reduce heat to low. Simmer until all water absorbed. 
            10. Saute cranberries and almonds in butter over medium heat in a small pan. Add to rice before serving.""", 
            
             """Spiced Fruit Tart Pie dough 1/2 pound dried figs 1/4 pound raisins 1/2 pint red wine 
             1 pound cooking apples 1 pound pears 4 ounces caster sugar 1 tablespoon flaked almonds 
             1/2 teaspoon powdered cloves 1/2 teaspoon ground mace 1/2 teaspoon powdered cinnamon 
             1/2 teaspoon powdered ginger Instructions: 
             1. Preheat oven to 350 degrees F. 
             2. Simmer figs and raisins in wine for 30 minutes. 
             3. Peel, core, and slice apples and pears. 
             4. Add the sugar, almonds, and spices to the figs and raisins and simmer until the apples and pears are soft. 
             5. Line a dish with pie dough. 
             6. Fill with the fruit mixture and bake for 1/2 hour or until pastry is cooked.""", 
             
             """Lemon Ice with Ginger and Lemon Cups 8 medium to large lemons 1 1/2 cups sugar 2 cups water 2-inch piece fresh ginger, 
             peeled and sliced Crushed ice 1 lemon, zest finely grated Fresh mint leaves, for garnish Instructions: 
             1. To make the lemon cups, trim a thin slice off the bottom of each lemon so they can stand upright. 
             2. Hollow out the insides of the lemons using a teaspoon, taking care not to damage the lemon shell. 
             3. Reserve the chopped lemon pulp for later use. 
             4. Freeze the lemon cups in an egg carton for at least 1 hour or overnight. 
             5. To make the ginger syrup, combine sugar, water, and ginger in a pot over medium heat. 
             6. Simmer and stir for 2 minutes until the sugar is dissolved and the ginger is fragrant. 
             7. Remove from heat, cover, and steep until cool.
             8. Fill a blender with crushed ice and pour in the reserved lemon juice, adding the ginger 
             syrup after removing the big pieces of ginger. 
             9. Blend until slushy and freeze for 1 hour to firm up. 
             10. Scoop the lemon ginger ice into the prepared lemon cups and garnish with mint leaves before serving.""", 
             
             """Tuna Provençal 5 tablespoons extra-virgin olive oil 1 teaspoon ground cumin, plus additional 1 teaspoon fennel seeds, 
             plus additional 1 clove garlic, peeled and chopped 2 sprigs thyme, leaves only 1 lime, zest finely grated Freshly ground 
             black pepper 2 (1-pound) tuna steaks 2 Texas onions (spring onions), peeled, trimmed, and each cut into 8 wedges 1 bulb fennel, 
             trimmed and cut into 1/4-inch thick slices 2 small zucchini, trimmed and cut into 1/4-inch-thick slices 2 orange baby bell peppers,
             cored, seeded, deveined, and cut into 1/4-inch thick slices Salt 12 fava beans, peeled 8 cherry tomatoes, halved 1 seedless orange,
             peeled and cut in segments 1 lemon, zest finely grated, flesh peeled, and cut in segments 4 sprigs basil, leaves only, 
             washed 2 tablespoons Nicoise olives, pitted 1 tablespoon pistachios Vegetable oil Fleur de sel Instructions: 
             1. Combine olive oil, cumin, fennel seeds, garlic, thyme leaves, and lime zest in a bowl. Season with pepper to taste. 
             Rub the marinade all over tuna steaks and refrigerate for at least 30 minutes or up to 2 hours. 
             2. Heat olive oil in a large skillet over medium heat. Cook onions, fennel, cumin, and fennel seeds for 
             5 minutes without browning. Add zucchini and bell peppers, season with salt and pepper, 
             and continue cooking for another 5 minutes. 
             3. Heat additional olive oil in a small skillet over medium heat. Cook fava beans for 1 minute,
             then add cherry tomatoes, orange segments, lemon zest and segments, half of the basil leaves, olives, and pistachios. 
             4. Grill tuna steaks over high heat until medium rare, about 4 minutes per side. Let rest 
             for 5 minutes before slicing into 4 pieces. 5. Divide vegetables among 4 dinner plates and
             top each plate with remaining basil leaves and 2 tuna slices. Spoon citrus-fava mixture around the plate and serve immediately.""", 
            
             """Eggs with Peas and Shallots 2 tablespoons butter 1 large shallot, very thinly sliced 8 eggs Salt and freshly ground 
             black pepper 1 package soft herb-flavored cheese, crumbled (recommended: Boursin) 1 cup green peas Instructions: 
             1. Melt butter in a medium skillet over medium heat. 
             2. Add shallots and cook for 3-4 minutes. 
             3. Whisk eggs with salt and pepper and combine with shallots. 
             4. Stir in the cheese and peas. 
             5. Cook for 3-4 minutes until eggs are firming up but not yet dry. 
             6. Serve immediately."""]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu") # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
Untrained model predictions:
----------------------------
Pan-Seared Sole with Mustard Sauce and Zucchini FrittersRice Almandine with Cranberries 3 sole fillets Salt and pepper. 
            Olive oil, for sauteing White Wine Mustard Sauce, recipe follows Green olives, chopped Zucchini Fritters, recipe follows 
            Rice Almandine, recipe follows 1 cup white wine 1 cup yellow mustard 1 cup heavy cream ...3 zucchini Salt 1 egg, 
            beaten 1/2 cup Italian bread crumbs 1 clove garlic, minced Vegetable oil, for sauteing 1 cup long grain rice 
            1/8 cup dried cranberries 1/4 cup slivered almonds 1 tablespoon butter Instructions: 
            1. Remove skin from sole fillets with a knife. 
            2. Season each fillet with salt and pepper on both sides. 
            3. Heat olive oil in a large saute pan over high heat. Sear sole fillets until slightly brown and cooked through. 
            4. Top each fille